In [1]:
import os

import geopandas as gpd

notebook_path = os.getcwd()
if not notebook_path.endswith('/notebooks/legacy'):
    notebook_path = os.path.join(notebook_path, '/notebooks/legacy')

# Read the shapefile
directory = os.path.join(notebook_path, 'assets', 'Mumbwa', 'Pumping-Wells-North')
gdf = gpd.read_file(os.path.join(directory, 'Wells_south_for_import.shp'))

# Print the first few rows of the GeoDataFrame
gdf

,fid,Layer,Abs_type,Yield,Abstract,x,y,Pumprate,geometry
0,216.0,1,Hand pump,1.03170,44.56944,27.427134,-16.495373,-7.50,MULTIPOINT (27.42713 -16.49537)
1,217.0,1,Hand pump,0.57259,24.73589,27.766712,-16.199571,-7.50,MULTIPOINT (27.76671 -16.19957)
2,218.0,2,hand pump,2.17671,94.03387,27.197459,-16.787903,-7.50,MULTIPOINT (27.19746 -16.78790)
3,219.0,2,hand pump,0.20532,8.86982,27.223616,-16.784734,-7.50,MULTIPOINT (27.22362 -16.78473)
4,220.0,2,hand pump,1.59138,68.74762,27.201646,-16.722441,-7.50,MULTIPOINT (27.20165 -16.72244)
...,...,...,...,...,...,...,...,...,...
255,544.0,2,subm pump,0.20047,8.66030,27.364086,-16.521373,-8.66,MULTIPOINT (27.36409 -16.52137)
256,545.0,2,subm pump,0.26547,11.46830,27.381031,-16.502268,-11.47,MULTIPOINT (27.38103 -16.50227)
257,546.0,2,subm pump,1.19157,51.47582,27.938661,-16.261872,-51.48,MULTIPOINT (27.93866 -16.26187)
258,547.0,2,subm pump,1.30770,56.49264,27.725843,-16.221709,-56.49,MULTIPOINT (27.72584 -16.22171)


In [2]:
geojson = gdf.to_crs(epsg=4326).__geo_interface__
geojson

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'fid': 216.0,
    'Layer': '1',
    'Abs_type': 'Hand pump',
    'Yield': 1.0317,
    'Abstract': 44.56944,
    'x': 27.42713359,
    'y': -16.49537344,
    'Pumprate': -7.5},
   'geometry': {'type': 'MultiPoint',
    'coordinates': ((27.427133589372275, -16.495373440049704),)},
   'bbox': (27.427133589372275,
    -16.495373440049704,
    27.427133589372275,
    -16.495373440049704)},
  {'id': '1',
   'type': 'Feature',
   'properties': {'fid': 217.0,
    'Layer': '1',
    'Abs_type': 'Hand pump',
    'Yield': 0.57259,
    'Abstract': 24.73589,
    'x': 27.76671229,
    'y': -16.19957115,
    'Pumprate': -7.5},
   'geometry': {'type': 'MultiPoint',
    'coordinates': ((27.76671229282504, -16.199571149688815),)},
   'bbox': (27.76671229282504,
    -16.199571149688815,
    27.76671229282504,
    -16.199571149688815)},
  {'id': '2',
   'type': 'Feature',
   'properties': {'fid': 218.0,
    'Lay

In [3]:
geojson.keys()

dict_keys(['type', 'features', 'bbox'])

In [4]:
geojson['features'][0]

{'id': '0',
 'type': 'Feature',
 'properties': {'fid': 216.0,
  'Layer': '1',
  'Abs_type': 'Hand pump',
  'Yield': 1.0317,
  'Abstract': 44.56944,
  'x': 27.42713359,
  'y': -16.49537344,
  'Pumprate': -7.5},
 'geometry': {'type': 'MultiPoint',
  'coordinates': ((27.427133589372275, -16.495373440049704),)},
 'bbox': (27.427133589372275,
  -16.495373440049704,
  27.427133589372275,
  -16.495373440049704)}

In [5]:
import dataclasses

from morpheus.project.types.geometry import Point


@dataclasses.dataclass
class PumpingWell:
    type: str
    name: str
    geometry: Point
    layers: list[int]
    well_type: str
    sp_values: list[list[int]]
    is_excluded_from_calculation: bool

    @classmethod
    def from_geojson(cls, geojson):
        properties = geojson['properties']
        print(geojson['geometry'])
        return cls(
            type='wel',
            name=str(properties['fid']),
            geometry=Point((geojson['geometry']['coordinates'][0][0], geojson['geometry']['coordinates'][0][1]), 'Point'),
            layers=[int(properties['Layer']) - 1],
            well_type='puw',
            sp_values=[[properties['Pumprate']]],
            is_excluded_from_calculation=False,
        )

    def to_dict(self):
        return {
            'type': self.type,
            'name': self.name,
            'geometry': self.geometry.to_dict(),
            'layers': self.layers,
            'well_type': self.well_type,
            'sp_values': self.sp_values,
            'is_excluded_from_calculation': self.is_excluded_from_calculation,
        }

In [6]:
feature = geojson['features'][0]
pumping_well = PumpingWell.from_geojson(feature)
pumping_well.to_dict()

{'type': 'MultiPoint', 'coordinates': ((27.427133589372275, -16.495373440049704),)}


{'type': 'wel',
 'name': '216.0',
 'geometry': {'type': 'Point',
  'coordinates': (27.427133589372275, -16.495373440049704)},
 'layers': [0],
 'well_type': 'puw',
 'sp_values': [[-7.5]],
 'is_excluded_from_calculation': False}

In [8]:
pumping_wells = [PumpingWell.from_geojson(feature).to_dict() for feature in geojson['features']]
pumping_wells

{'type': 'MultiPoint', 'coordinates': ((27.427133589372275, -16.495373440049704),)}
{'type': 'MultiPoint', 'coordinates': ((27.76671229282504, -16.199571149688815),)}
{'type': 'MultiPoint', 'coordinates': ((27.197458912114172, -16.787902959596348),)}
{'type': 'MultiPoint', 'coordinates': ((27.223616311683628, -16.784734167488477),)}
{'type': 'MultiPoint', 'coordinates': ((27.201645660942575, -16.722440910681495),)}
{'type': 'MultiPoint', 'coordinates': ((27.21431109489918, -16.739758953030325),)}
{'type': 'MultiPoint', 'coordinates': ((27.2835832642945, -16.728902866781805),)}
{'type': 'MultiPoint', 'coordinates': ((27.26833304789777, -16.734589388150077),)}
{'type': 'MultiPoint', 'coordinates': ((27.255926092185174, -16.71184330267699),)}
{'type': 'MultiPoint', 'coordinates': ((27.28487565551456, -16.684444608811678),)}
{'type': 'MultiPoint', 'coordinates': ((27.307542527203736, -16.668086603624328),)}
{'type': 'MultiPoint', 'coordinates': ((27.441728225713806, -16.570483445646452),)}

[{'type': 'wel',
  'name': '216.0',
  'geometry': {'type': 'Point',
   'coordinates': (27.427133589372275, -16.495373440049704)},
  'layers': [0],
  'well_type': 'puw',
  'sp_values': [[-7.5]],
  'is_excluded_from_calculation': False},
 {'type': 'wel',
  'name': '217.0',
  'geometry': {'type': 'Point',
   'coordinates': (27.76671229282504, -16.199571149688815)},
  'layers': [0],
  'well_type': 'puw',
  'sp_values': [[-7.5]],
  'is_excluded_from_calculation': False},
 {'type': 'wel',
  'name': '218.0',
  'geometry': {'type': 'Point',
   'coordinates': (27.197458912114172, -16.787902959596348)},
  'layers': [1],
  'well_type': 'puw',
  'sp_values': [[-7.5]],
  'is_excluded_from_calculation': False},
 {'type': 'wel',
  'name': '219.0',
  'geometry': {'type': 'Point',
   'coordinates': (27.223616311683628, -16.784734167488477)},
  'layers': [1],
  'well_type': 'puw',
  'sp_values': [[-7.5]],
  'is_excluded_from_calculation': False},
 {'type': 'wel',
  'name': '220.0',
  'geometry': {'type'

In [9]:
import simplejson as json

with open(os.path.join(directory, 'Wells_south_for_import.json'), 'w') as f:
    json.dump(pumping_wells, f, ignore_nan=True)